In [15]:
import pandas as pd

data = pd.read_csv("data/spotify-tracks-genre.csv")

length = data["Unnamed: 0"].count()

In [16]:
def remove_colon(data):
  columns = [
  "energy", "danceability", "loudness", "speechiness",
  "acousticness", "instrumentalness",
  "liveness", "valence", "tempo"]
  
  for column in columns:
    data[column] = (data[column].str.replace(",", ".")).astype("float64")
    
  return data

data = remove_colon(data)

1. Qual o ritmo mais popular?

In [17]:
popularity = {}

genres = data["track_genre"].unique()

bigger_total_genre = ""
bigger_total_popularity = 0

bigger_mean_genre = ""
bigger_mean_popularity = 0

bigger_median_genre = ""
bigger_median_popularity = 0

for genre in genres:
  all_musics_genre = data[data["track_genre"] == genre]
  sum_popularity = all_musics_genre["popularity"].sum()
  mean_popularity = round(all_musics_genre["popularity"].mean(), 2)
  median_popularity = round(all_musics_genre["popularity"].median(), 2)
  
  if sum_popularity > bigger_total_popularity:
    bigger_total_genre = genre
    bigger_total_popularity = sum_popularity
    
  if mean_popularity > bigger_mean_popularity:
    bigger_mean_genre = genre
    bigger_mean_popularity = mean_popularity
    
  if median_popularity > bigger_median_popularity:
    bigger_median_genre = genre
    bigger_median_popularity = median_popularity
    

print("Total", bigger_total_genre, bigger_total_popularity)
print("Mean", bigger_mean_genre, bigger_mean_popularity)
print("Median", bigger_median_genre, bigger_median_popularity)

Total pop-film 59283
Mean pop-film 59.28
Median pop 66.0


2. Qual a probabilidade de uma música ser popular (>P70) se ela for do ritmo mais popular?

Qual a proporção que as músicas do ritmo mais popular representam dentre as músicas populares?


In [18]:
all_pop_film = data[data["track_genre"] == bigger_total_genre]
length_pop_film = all_pop_film["track_genre"].count()

p70 = data["popularity"].quantile(0.70)
p70_pop_film = all_pop_film[all_pop_film["popularity"] > p70]
length_p70_pop_film = p70_pop_film["track_genre"].count()

print(f"a. {((length_p70_pop_film / length_pop_film) * 100):.2f}%")

popularity_musics = data[data["popularity"] > p70]
length_popularity_musics = popularity_musics["popularity"].count()

print(f"b. {((length_pop_film / length_popularity_musics) * 100):.2f}")

a. 94.60%
b. 3.06


3. Qual a probabilidade de uma música não ser explícita ou não possuir dançabilidade entre 0,5 e 0,6, nem ter menos de 3 minutos de duração.

In [19]:
explicits = data[data["explicit"] == False]
length_explicits = explicits["explicit"].count()
prob_explicits = length_explicits / length

danceabilities = data[data["danceability"] > 0.5]
danceabilities = danceabilities[danceabilities["danceability"] < 0.6]
length_danceabilities = length - danceabilities["danceability"].count()
prob_danceabilities = length_danceabilities / length

two_and = danceabilities[danceabilities["explicit"] == False]
length_two_and = length_explicits - two_and["explicit"].count() # 81545
prob_tow_and = length_two_and / length 

duration = data[data["duration_ms"] >= 180000]
prob_duration = duration["duration_ms"].count() / length

print(f"{((((prob_explicits + prob_danceabilities) - prob_tow_and) * prob_duration) * 100):.2f}%")


70.59%


4. Dentre os traços (duration_ms, danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo) qual é mais provável de indicar uma música popular caso:

a. esteja abaixo do primeiro quartil?

b. esteja acima do terceiro quartil?

c. esteja dentro de ±25% do segundo quartil?


In [20]:
def define_probability(question, quantile_value, sinal, between):
  keys = [
  "duration_ms", "danceability", "energy",
  "key", "loudness", "speechiness",
  "acousticness", "instrumentalness",
  "liveness", "valence", "tempo"]
  
  bigger_name = "" 
  bigger_prob = 0
  for key in keys:
    if not between:
      quantile = data[key].quantile(quantile_value)
      condition = data[key] < quantile if sinal == "<" else data[key] > quantile
      quantile_data = data[condition]
    
    if between:
      first_quantile = data[key].quantile(0.25)
      second_quantile = data[key].quantile(0.5)
      third_quarile = data[key].quantile(0.75)

      limit_down = second_quantile - 0.25 * (second_quantile - first_quantile)
      limit_up = second_quantile + 0.25 * (third_quarile - second_quantile)
      
      condition_up = data[key] < limit_up
      quantile_data = data[condition_up]
      
      condition_down = quantile_data[key] > limit_down
      quantile_data = quantile_data[condition_down]
          
    popularity_music = quantile_data[quantile_data["popularity"] > p70]
    
    length_popularity_music = popularity_music["Unnamed: 0"].count()
    length_quantile = quantile_data["Unnamed: 0"].count()
    prob_first =  length_popularity_music / length_quantile if length_quantile != 0 else 0
    prob_first *= 100
    
    if prob_first > bigger_prob:
      bigger_name = key
      bigger_prob = prob_first
      
  print(f"{question}. {bigger_name}: {bigger_prob:.2f}% ")
    
define_probability("a", 0.25, "<", False)
define_probability("b", 0.75, ">", False)
define_probability("b", 0, ">", True)

a. speechiness: 32.05% 
b. loudness: 30.30% 
b. duration_ms: 32.94% 
